## Evaluation of basic n-gram model
This notebook uses the file with the probabilities created by the model in basic-ngram-model2.ipynb to generate a text. By using this code, you can generate a song without having to train the model.

In [4]:
import numpy as np
import tensorflow as tf

def top_k_sampling(conditional_probability, target_words, k):
    n_conds = len(conditional_probability)
    #conditional_probability = np.array(conditional_probability)
    k = min(n_conds,k)
    top_k_probabilities, top_k_indices= tf.math.top_k(conditional_probability, k=k, sorted=True)
    top_k_indices = np.asarray(top_k_indices).astype("int32")
    top_k_redistributed_probability=tf.nn.softmax(np.log(top_k_probabilities))
    top_k_redistributed_probability = np.asarray(top_k_redistributed_probability).astype("float32")
    sampled_index = np.random.choice(top_k_indices, p=top_k_redistributed_probability)
    sampled_token = target_words[sampled_index]
    return sampled_token

def generate_text(conditional_dict, context_queue, token_count: int, k = 5):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        result = [] + context_queue
        for _ in range(token_count):
            context_string = '_'.join(context_queue)
            conditional_prob = conditional_dict[context_string]
            probs = conditional_prob['probabilities']
            words = conditional_prob['target_words']
            obj = top_k_sampling(probs, words, k)
            if obj == 'NEWLINE':
                result.append('\n')
            else:
                result.append(obj)
            context_queue.pop(0)
            context_queue.append(obj)
        return ' '.join(result).replace(' \' ', '\'')
    
def create_song(all_conds,genre, sentiment, context_queue, token_count, k = 5):
    conddict = all_conds[genre][sentiment]
    return generate_text(conddict, context_queue, token_count, k)

In [2]:
import json
with open('../input/ngram-lyric-probabilities/ngrams_probabilities.json', 'r') as infile:
    conditional_probs_per_input = json.load(infile)

In [10]:
print(create_song(conditional_probs_per_input, 'Pop', 'Positive',['i', 'want'], 210, 15))

i want your love 
 all the time is running out of your heart ? 
 why don't you tell me what you need me man i don't go wrong 
 don't have to give them a kiss 
 just the same 
 so don't you just don't be found 
 i'm the one in a lifetime 
 it's not hard it's time for you 
 so much to you and you're so beautiful won't know 
 the only thing 
 i got money to share 
 and the sun to shine 
 and i'm so far away 
 so i will do for you 
 but i'm on fire 
 to get you back 
 so i'm gonna 
 stomp you straight 
 don't wanna know you got a way to love you more 
 i love you 
 and i'm gonna give you a little bit of my life 
 i'm not your fault 
 meanwhile the taxis are calling 
 i'll be there 
 but you're gonna take you home 
 but there
